##Librerias

In [ ]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from mlxtend.preprocessing import TransactionEncoder
from ipywidgets import interact
from sklearn import neighbors
import os
os.getcwd()

'/content'

##Limpieza y transformación

In [138]:
!git clone "https://github.com/AngelicaCeDi/Analitica-3-Caso-Marketing.git"

fatal: destination path 'Analitica-3-Caso-Marketing' already exists and is not an empty directory.


In [139]:
os.chdir('/content')

In [140]:
db_movies="/content/Analitica-3-Caso-Marketing/db_movies"
conn= sql.connect(db_movies)
cur=conn.cursor()

In [141]:
#Exploramos que tablas que hay en el sql. 
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',), ('df_rt',), ('df_mov',), ('mov_data',)]

In [142]:
#Las tablas las llevamos a python.
movies = pd.read_sql("SELECT* FROM movies", conn)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [143]:
ratings = pd.read_sql("SELECT* FROM ratings", conn)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Limpieza y transformación

Limpieza y transformación



In [144]:
#Se hace una copia de la tablas originales. 
df_mov=movies.copy()
df_rt=ratings.copy()

In [145]:
#Vemos el tamaño de las tablas.
print(df_mov.shape)
print('----------------')
print(df_rt.shape)

(9742, 3)
----------------
(100836, 4)


In [146]:
#Tipos de variable de cada columna.
print(df_mov.dtypes)
print('----------------')
print(df_rt.dtypes)

movieId     int64
title      object
genres     object
dtype: object
----------------
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


In [147]:
#Vemos si las tablas tienen valores nulos. 
print(df_mov.isnull().sum())
print('----------------')
print(df_rt.isnull().sum()) 

movieId    0
title      0
genres     0
dtype: int64
----------------
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [148]:
#Vemos si las tablas tienen datos duplicados
print(df_mov.duplicated().sum())
print('----------------')
print(df_rt.duplicated().sum()) 

0
----------------
0


In [149]:
#En el dataframe df_rt se cambia el formato de timestamp por uno de fecha para hacer su manejo mas cómodo. 
df_rt['time'] =pd.to_datetime(df_rt['timestamp'], unit='s')
df_rt.drop(columns=["timestamp"],inplace=True)
df_rt.head()

,userId,movieId,rating,time
0,1,1,4.0,2000-07-30 18:45:03
1,1,3,4.0,2000-07-30 18:20:47
2,1,6,4.0,2000-07-30 18:37:04
3,1,47,5.0,2000-07-30 19:03:35
4,1,50,5.0,2000-07-30 18:48:51


In [150]:
# Se separan los generos para mejor manejo del dataframe 
genres=df_mov['genres'].str.split('|')
transactionencoder= TransactionEncoder()
genres = transactionencoder.fit_transform(genres)
genres = pd.DataFrame(genres, columns = transactionencoder.columns_)
genres  = genres.astype(int)
df_mov = movies.drop(['genres'], axis = 1) 
df_mov =pd.concat([df_mov, genres],axis=1)
df_mov

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
#Obtenemos el año de premiere de cada pelicula de su titulo.
df_mov['movyear'] = df_mov['title'].str[-5:-1]

In [152]:
df_mov=df_mov[df_mov["movyear"].str.contains ('r On') == False ]

In [153]:
#Se crea una tabla en general para trabajar los modelos que necesiten datos cruzados.
mov_data=pd.merge(df_mov,df_rt, on='movieId')
mov_data

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Mystery,Romance,Sci-Fi,Thriller,War,Western,movyear,userId,rating,time
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1995,1,4.0,2000-07-30 18:45:03
1,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1995,5,4.0,1996-11-08 06:36:02
2,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1995,7,4.5,2005-01-25 06:52:26
3,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1995,15,2.5,2017-11-13 12:59:30
4,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,1995,17,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100827,193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,2017,184,4.0,2018-09-16 14:44:42
100828,193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,2017,184,3.5,2018-09-16 14:52:25
100829,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2017,184,3.5,2018-09-16 14:56:45
100830,193587,Bungo Stray Dogs: Dead Apple (2018),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,2018,184,3.5,2018-09-16 15:00:21


In [154]:
#Cantidad de calificaciones que tiene cada usuario.
numusers=pd.DataFrame(df_rt.groupby(('userId'))['rating'].count().sort_values(ascending=False))
numusers

,rating
userId,
414,2698
599,2478
474,2108
448,1864
274,1346
...,...
442,20
569,20
320,20


In [155]:
fig=px.histogram(numusers, x='rating', title='Histograma usuarios')
fig.show() #La mayoría de usuarios calificaron muy pocas peliculas y solo uno calificó 2698 peliculas lo que podrian considerar datos atipicos sesgando las recomendaciones.

In [156]:
#Cantidad de calificaciones por pelicula  
nummov=pd.DataFrame(df_rt.groupby(('movieId'))['rating'].count().sort_values(ascending=False))
nummov

,rating
movieId,
356,329
318,317
296,307
593,279
2571,278
...,...
4093,1
4089,1
58351,1


In [157]:
#Cantidad de calificaciones que tiene cada pelicula.
fig=px.histogram(nummov, x='rating', title='Histograma movies')
fig.show() # Muchas peliculas tienen muy pocas calificaciones que podrian sesgar las recomendaciones. 

In [158]:
#Debido a lo anterior, se requiere que las tablas generadas tengas datos que no sesgen los modelos, en este caso se limitaran a los uruarios que tienen menos de 1500 peliculas calificadas y 
#las peliculas que tienen mas de 3 calificaciones.
newuser = numusers.drop(numusers[numusers['rating']>1500].index).reset_index()
newmovs = nummov.drop(nummov[nummov['rating']<3].index).reset_index()


In [159]:
#Hacemos una tabla para actualizar la información el Id de los usuarios que tienen menos de 1500 peiculas calificadas.
dicuser= newuser.drop(columns=['rating'])
dicuser

,userId
0,274
1,610
2,68
3,380
4,606
...,...
601,442
602,569
603,320
604,576


In [160]:
#Hacemos una tabla para actualizar la información el Id de las peliculas que tienen mas de 3 calificaciones.
dicmov= newmovs.drop(columns=['rating'])
dicmov

,movieId
0,356
1,318
2,296
3,593
4,2571
...,...
4975,2177
4976,869
4977,6790
4978,3064


In [161]:
#Realizamos un merge para tener la nueva indormación en las tablas, peliculas con mas de 3 calificaciones y usuarios que han calificado menos de 1500 peliculas.
mov_data=pd.merge(mov_data,dicmov,how='right', on='movieId')
mov_data=pd.merge(mov_data,dicuser,how='right', on='userId')
df_mov=pd.merge(df_mov,dicmov,how='right', on='movieId')
df_rt=pd.merge(df_rt,dicuser,how='right',on='userId')

In [162]:
#Verificamos duplicados o nulos, ya que al unir tablas aveces se presentan estos imperfectos.
print(mov_data.isnull().sum())
print('----------------')
print(mov_data.duplicated().sum()) #Observamos que ninguna de las tablas presentadas tiene presencia de nulos, registros extraños o registros duplicados.
print('----------------')
print(df_mov.isnull().sum())
print('----------------')
print(df_mov.duplicated().sum())
print('----------------')
print(df_rt.isnull().sum())
print('----------------')
print(df_rt.duplicated().sum())

movieId               0
title                 0
(no genres listed)    0
Action                0
Adventure             0
Animation             0
Children              0
Comedy                0
Crime                 0
Documentary           0
Drama                 0
Fantasy               0
Film-Noir             0
Horror                0
IMAX                  0
Musical               0
Mystery               0
Romance               0
Sci-Fi                0
Thriller              0
War                   0
Western               0
movyear               0
userId                0
rating                0
time                  0
dtype: int64
----------------
0
----------------
movieId               0
title                 1
(no genres listed)    1
Action                1
Adventure             1
Animation             1
Children              1
Comedy                1
Crime                 1
Documentary           1
Drama                 1
Fantasy               1
Film-Noir             1
Horror         

In [163]:
#Se guardan las 3 tablas generadas en la base de datos de SQL.
df_rt.to_sql("df_rt", conn, if_exists="replace")
df_mov.to_sql("df_mov", conn, if_exists="replace")
mov_data.to_sql("mov_data", conn, if_exists="replace")

87276

In [164]:
#Se confirma que si se hayan guardado las tablas correctamente.
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',), ('df_rt',), ('df_mov',), ('mov_data',)]

## Análisis exploratorio

In [165]:
#Cantidad de peliculas. 
pd.read_sql("""select count(title) as cantidad_registros_movies from movies""", conn) 

,cantidad_registros_movies
0,9742


In [166]:
#Cantidad de registros.
pd.read_sql("""select count(userId) as cantidad_registros_ratings from ratings""", conn)

,cantidad_registros_ratings
0,100836


In [167]:
# Número de peliculas por géneros.
pd.read_sql("""select movieId, genres, count(title) from movies group by genres order by count(title) desc """, conn)

,movieId,genres,count(title)
0,14,Drama,1053
1,5,Comedy,946
2,72,Comedy|Drama,435
3,3,Comedy|Romance,363
4,17,Drama|Romance,349
...,...,...,...
946,3745,Action|Adventure|Animation|Children|Sci-Fi,1
947,558,Action|Adventure|Animation|Children|Fantasy,1
948,84944,Action|Adventure|Animation|Children|Comedy|Wes...,1
949,85261,Action|Adventure|Animation|Children|Comedy|Sci...,1


In [168]:
#Número de usuarios que calificaron títulos.
pd.read_sql("""select count(distinct(userId)) as cantidad_usuarios from ratings""", conn)

,cantidad_usuarios
0,610


In [169]:
df_mov

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movyear
0,356,Forrest Gump (1994),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1994
1,318,"Shawshank Redemption, The (1994)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1994
2,296,Pulp Fiction (1994),0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1994
3,593,"Silence of the Lambs, The (1991)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1991
4,2571,"Matrix, The (1999)",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4975,2177,Family Plot (1976),0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1976
4976,869,Kansas City (1996),0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1996
4977,6790,Avalon (2001),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2001
4978,3064,Poison Ivy: New Seduction (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1997


In [170]:
#Numero de titulos por genero (hay titulos que comparten generos)

pd.DataFrame(df_mov.loc[:,~df_mov.columns.isin(['title','movieId','movyear'])].sum().sort_values(ascending=False))

,0
Drama,2206.0
Comedy,2012.0
Action,1148.0
Thriller,1137.0
Romance,891.0
Adventure,814.0
Crime,693.0
Sci-Fi,600.0
Fantasy,455.0
Horror,441.0


In [171]:
#Top 10 peliculas más calificadas.
mt= pd.read_sql('''select title, count(*) as conteo from mov_data group by title order by conteo desc limit 10''', conn)

dfmt  = go.Bar( x=mt.title,y=mt.conteo, text=mt.conteo, textposition="outside")
Layout=go.Layout(title="<b>Top 10 movies<b>",xaxis={'title':'title'},yaxis={'title':'# Ratings'})
go.Figure(dfmt,Layout)

#Las peliculas que mas fueron calificadas independiente de si tuvo buenas o malas calificaciones fueron Forrest Gump (1994), Shawnshank Redemption(1994) y Pulp Fiction (1994), 
#llama la atención que todas las peliculas tienen el mismo año de premiere y en la epoca de los años 90s.

In [172]:
# Número de lanzamientos por año.
base = mov_data.groupby(['movyear'])[['movieId']].count().sort_values('movyear', ascending= True).reset_index()

base  = go.Bar( x=base.movyear, y= base.movieId, textposition="outside")
Layout=go.Layout(title="<b>Premieres per year<b>",xaxis={'title':'title'},yaxis={'title':'count'})
go.Figure(base,Layout)

#Como s epuede evidenciar en el gráfico, el año con más número de lanzamientos es 1995 seguido de 1994 y 1999. 
#Esto tiene sentido pues la decada de los 90s fue en donde se presento la mayor producción de cine del mundo pues 
#nacieron muchos directores por esta epoca y se probaron muchisimos estilos y metodos en las peliculas ya que iba de la mano con la tecnología que surgía en esos años.

In [173]:
base2= pd.read_sql('''select rating,count(*) as conteo from mov_data group by rating order by conteo desc''', conn)

base2  = go.Bar( x=base2.rating, y= base2.conteo, textposition="outside")
Layout=go.Layout(title="<b>Premieres per year<b>",xaxis={'title':'rating'},yaxis={'title':'count'})
go.Figure(base2,Layout)

#La calificación que más se colocó en los ratings es el 4 seguido de un 3 y un 5, 
#esto quiere decir que los usuarios en general se han sentido satisfechos con las peliculas que ofrece la plataforma online.  